In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pprint
from IPython import display
from pathlib import *
import pandas as pd
import xarray as xr
from bokeh.palettes import all_palettes
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from climatools.plot.plot import *
from climatools.atm import *

from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *
from exp.nb_03 import *
from exp.nb_04 import *

output_notebook()

Loading BokehJS ...

In [2]:
! pwd

/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev


In [3]:
PARAM = LBLnewBestfitSWParam(**{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})

In [4]:
runner = LBLnewBestfitSWRun(path=PATH_FORTRAN/PARAM.to_path(), param=PARAM)

In [5]:
# Uncomment this cell when testing this notebook on the spot.
#PARAM = LBLnewBestfitSWParam(band=10, vmin=4400, vmax=8200, nv=10000, dv=0.001,
#                     molecule='h2o', conc='atmpro',
#                     ref_pts=[(30, 250), (300, 250)], ng_refs=[3, 9], ng_adju=[0, 0],
#                     wgt=[(.95, .9, .7), (.45, .45, .45, .45, .5, .55, .7, .8, .95)],
#                     cosz=.2588, rsfc=0,
#                     klin=3e-25,
#                     option_k_lookup=0,
#                     atmpro='trp',
#                     commitnumber='79f2292')
#runner = LBLnewBestfitSWRun(path=PATH_FORTRAN/PARAM.to_path(), param=p)

In [6]:
md = LBLnewBestfitSWModelData.from_runner(runner)

In [7]:
np.random.seed(30)

## CRD heating rate for each $g$

In [8]:
pltdata = [{'srs':md.crd_heat['heatrg'].sel(g=g),
            'label':f'{int(g)}',
            'color':np.random.choice(all_palettes['Category20'][15]),
            'alpha':np.random.uniform(.8, 1.),
            'line_width':np.random.uniform(2, 7),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in md.crd_heat['g']]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', prange=(50, 1050))
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heat', prange=(1e-2, 200))
show(gridplot(fig_linp, fig_logp, ncols=2))

## WGT heating rate for each $g$

In [9]:
pltdata = [{'srs':md.wgt_heat['heatrg'].sel(g=g),
            'label':f'{int(g)}',
            'color':np.random.choice(all_palettes['Category20'][15]),
            'alpha':np.random.uniform(.8, 1.),
            'line_width':np.random.uniform(2, 7),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in md.wgt_heat['g']]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heatg', prange=(50, 1050))
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heatg', prange=(1e-2, 200))
show(gridplot(fig_linp, fig_logp, ncols=2))

## (CRD vs WGT)  heating rate for each $g$

In [10]:
for g in md.wgt_heat['g']:
    pltdata = [{'srs':md.wgt_heat['heatrg'].sel(g=g),
                'label':f'WGT g={g}',
                'color':all_palettes['Set1'][9][0],
                'alpha':.5,
                'line_width':6,
                'line_dash':'dotdash'},
               {'srs':md.crd_heat['heatrg'].sel(g=g),
                'label':f'CRD g={g}',
                'color':all_palettes['Set1'][9][1],
                'alpha':1,
                'line_width':3,
                'line_dash':'solid'}]

    fig_liny = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='linear', prange=(50, 1050))
    fig_logy = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='log', prange=(1e-2, 200))
    show(gridplot(fig_liny, fig_logy, ncols=2))

## Bands-total heating 

In [11]:
pltdata = [{'srs':md.wgt_heat['heatrg'].sum('g'),
            'label':f'WGT',
            'color':all_palettes['Category10'][3][1],
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])},
           {'srs':md.crd_heat['heatrg'].sum('g'),
            'label':f'CRD',
            'color':all_palettes['Category10'][3][0],
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])}]

fig_lin = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', prange=(50, 1050))
fig_log = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', y_axis_type='log', prange=(1e-2, 200))
show(gridplot(fig_lin, fig_log, ncols=2))

## Flux for each g

In [12]:
dwgt = md.wgt_flux['fldg'].isel(pressure=[0,-1]).to_pandas()
dcrd = md.crd_flux['fldg'].isel(pressure=[0,-1]).to_pandas()
df = pd.concat([dwgt, dcrd, dwgt - dcrd], axis=0, keys=['WGT', 'CRD', 'WGT - CRD'])
df['band-total'] = df.sum(axis=1)
df

g                              1             2         3             4  \
          pressure                                                       
WGT       0.0       1.301068e-01  4.771250e-01  1.778085  4.191768e-01   
          1013.0    8.736685e-22  5.717573e-09  0.008218  3.855535e-09   
CRD       0.0       1.301068e-01  4.771250e-01  1.778085  4.191768e-01   
          1013.0    3.632803e-11  1.017362e-03  0.265054  4.520092e-06   
WGT - CRD 0.0       0.000000e+00  0.000000e+00  0.000000  0.000000e+00   
          1013.0   -3.632803e-11 -1.017356e-03 -0.256836 -4.516237e-06   

g                          5         6         7          8          9  \
          pressure                                                       
WGT       0.0       1.405519  4.444786  9.482400  17.564948  29.643659   
          1013.0    0.000990  0.468396  4.231143  12.815492  26.357019   
CRD       0.0       1.405519  4.444786  9.482400  17.564948  29.643659   
          1013.0    0.012975  0.714662  4.232764  12.567436  26.056151   
WGT - CRD 0.0       0.000000  0.000000  0.000000   0.000000   0.000000   
          1013.0   -0.011985 -0.246266 -0.001622   0.248056   0.300868   

g                          10         11         12  band-total  
          pressure                                               
WGT       0.0       39.715786  48.576952  288.78121  442.419754  
          1013.0    38.092029  47.847478  288.38302  418.203785  
CRD       0.0       39.715786  48.576952  288.78121  442.419754  
          1013.0    37.947237  47.805826  288.33960  417.942727  
WGT - CRD 0.0        0.000000   0.000000    0.00000    0.000000  
          1013.0     0.144792   0.041652    0.04342    0.261057

## Bands-total flux

In [13]:
pltdata = [{'srs':md.wgt_flux['fldg'].sum('g'),
            'label':'WGT',
            'color':all_palettes['Category10'][10][3],
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])},
           {'srs':md.crd_flux['fldg'].sum('g'),
            'label':'CRD',
            'color':all_palettes['Category10'][10][0],
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])}]

fig_lin = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='flux', prange=(50, 1050))
fig_log = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='flux', prange=(1e-2, 200), y_axis_type='log')
show(gridplot(fig_lin, fig_log, ncols=2))

# fin

In [14]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')